In [7]:
from omnibelt import toposort
from itertools import product
import numpy as np
import torch
from causalbenchmark.novo.simulation.models import Bernoulli, ConditionalBernoulli, BernoulliNetwork

In [2]:
nodes = [
    {
        "name": "Player Mortality",
        "type": "outcome",
        "observed": True,
        "values": ["alive", "deceased"],
        "parents": ["Physical Fitness", "Quality of Medical Care"]
    },
    {
        "name": "Physical Training Intensity",
        "type": "treatment",
        "observed": True,
        "values": ["low/moderate", "high"],
        "parents": ["Age of the Player"]
    },
    {
        "name": "Health Monitoring Practices",
        "type": "treatment",
        "observed": True,
        "values": ["inadequate", "adequate"],
        "parents": ["Age of the Player"]
    },
    {
        "name": "Age of the Player",
        "type": "confounder",
        "observed": True,
        "values": ["younger", "older"],
        "parents": []
    },
    {
        "name": "Physical Fitness",
        "type": "mediator",
        "observed": True,
        "values": ["poor", "good"],
        "parents": ["Physical Training Intensity"]
    },
    {
        "name": "Quality of Medical Care",
        "type": "mediator",
        "observed": True,
        "values": ["low", "high"],
        "parents": ["Health Monitoring Practices"]
    }
]


In [8]:
varvals = {node['name']: node['values'] for node in nodes}
varparents = {node['name']: node['parents'] for node in nodes}
varorder = toposort(varparents)
nodes = sorted(nodes, key=lambda x: varorder.index(x['name']))
print(varorder)

['Age of the Player', 'Physical Training Intensity', 'Health Monitoring Practices', 'Physical Fitness', 'Quality of Medical Care', 'Player Mortality']


In [57]:

template = '{index}. What is the probability that {outcome}{parents}?'
# val_template = '{name} = {val}'
# parentval_template = val_template
# parent_template = ' given that ({parents})'
# separator = ') and ('

template = '{index}. What is the probability that {outcome}{parents}?'
val_template = '"{name}" is "{val}" (rather than "{otherval}")'
parentval_template = '"{name}" is "{val}"'
parent_template = ' given that {parents}'
separator = ' and '

template = '{index}. {parents}what is the probability that {outcome}?'
val_template = '"{name}" is "{val}" (rather than "{otherval}")'
parentval_template = '"{name}" is "{val}"'
parent_template = 'when {parents}: '
separator = ' and '

In [58]:
def generate_terms(nodes):
    index = 1
    for node in nodes:
        name = node['name']
        parents = node['parents']
        vals = node['values']
        
        for parentvals in product(*[varvals[parent] for parent in parents]):
            yield template.format(
                index=index,
                outcome=val_template.format(name=name, val=vals[1], otherval=vals[0]),
                parents=parent_template.format(parents=separator.join([parentval_template.format(name=parent, val=parentval) for parent, parentval in zip(parents, parentvals)])) if len(parents) else ''
            )
            index += 1
            
        # for val in vals:
        #     if len(parents):
        #         for parentvals in product(*[varvals[parent] for parent in parents]):
        #             yield template.format(
        #                 index=index,
        #                 outcome=val_template.format(name=name, val=val),
        #                 parents=parent_template.format(parents=separator.join([val_template.format(name=parent, val=parentval) for parent, parentval in zip(parents, parentvals)])) if len(parents) else ''
        #             )
        #             index += 1
        #     else:
        #         yield template.format(
        #                 index=index,
        #             outcome=f'{name} = {val}',
        #             parents=''
        #         )
        #         index += 1
        

In [59]:
terms = list(generate_terms(nodes))
print('\n'.join(terms))

1. what is the probability that "Age of the Player" is "older" (rather than "younger")?
2. when "Age of the Player" is "younger": what is the probability that "Physical Training Intensity" is "high" (rather than "low/moderate")?
3. when "Age of the Player" is "older": what is the probability that "Physical Training Intensity" is "high" (rather than "low/moderate")?
4. when "Age of the Player" is "younger": what is the probability that "Health Monitoring Practices" is "adequate" (rather than "inadequate")?
5. when "Age of the Player" is "older": what is the probability that "Health Monitoring Practices" is "adequate" (rather than "inadequate")?
6. when "Physical Training Intensity" is "low/moderate": what is the probability that "Physical Fitness" is "good" (rather than "poor")?
7. when "Physical Training Intensity" is "high": what is the probability that "Physical Fitness" is "good" (rather than "poor")?
8. when "Health Monitoring Practices" is "inadequate": what is the probability tha

In [ ]:
invprobabilities = {
  1: [0.7, 0.9],
  2: [0.6, 0.8],
  3: [0.2, 0.4],
  4: [0.4, 0.6],
  5: [0.1, 0.3],
  6: [0.2, 0.4],
  7: [0.3, 0.5],
  8: [0.8, 0.9],
  9: [0.2, 0.4],
  10: [0.05, 0.2],
  11: [0.2, 0.5],
  12: [0.95, 0.99],
  13: [0.98, 1.0]
}

In [ ]:
# Probabilities: 
1: [0.6, 0.7]
2: [0.3, 0.4]
3: [0.6, 0.7]
4: [0.7, 0.8]
5: [0.8, 0.9]
6: [0.7, 0.8]
7: [0.3, 0.4]
8: [0.2, 0.3]
9: [0.7, 0.8]
10: [0.8, 0.9]
11: [0.3, 0.4]
12: [0.2, 0.3]
13: [0.1, 0.2]